In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3418, 0.3126, 0.3224], std=[0.1627, 0.1632, 0.1731])
])

In [ ]:
device = "cuda:0"

In [ ]:
trainset = torchvision.datasets.ImageFolder("../input/data-update/UPDATE DATA", transform = preprocess)

In [ ]:
train, test = torch.utils.data.random_split(trainset, [100,42],generator=torch.Generator().manual_seed(42))
train_loader = torch.utils.data.DataLoader(train, batch_size=500, shuffle=True, drop_last=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=500, shuffle=True, drop_last=False)

In [ ]:
accuracy = 0

In [ ]:
class SmallCNN(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.conv1 = nn.Conv2d(3, 16, 3, 1, padding=1)
        self.conv_bn_1 = nn.BatchNorm2d(16)
        torch.nn.init.xavier_normal_(self.conv1.weight)
        torch.nn.init.zeros_(self.conv1.bias)
        
        self.conv2 = nn.Conv2d(16, 32, 3, 1, padding=1)
        self.conv_bn_2 = nn.BatchNorm2d(32)
        torch.nn.init.xavier_normal_(self.conv2.weight)
        torch.nn.init.zeros_(self.conv2.bias)
        
        self.conv3 = nn.Conv2d(32, 64, 3, 1, padding=1)
        self.conv_bn_3 = nn.BatchNorm2d(64)
        torch.nn.init.xavier_normal_(self.conv3.weight)
        torch.nn.init.zeros_(self.conv3.bias)
        
        self.conv4 = nn.Conv2d(64, 128 , 3, 1, padding=1)
        self.conv_bn_4 = nn.BatchNorm2d(128)
        torch.nn.init.xavier_normal_(self.conv4.weight)
        torch.nn.init.zeros_(self.conv4.bias)
        
        self.conv5 = nn.Conv2d(128, 256 , 3, 1, padding=1)
        self.conv_bn_5 = nn.BatchNorm2d(256)
        torch.nn.init.xavier_normal_(self.conv5.weight)
        torch.nn.init.zeros_(self.conv5.bias)
        
        self.conv6 = nn.Conv2d(256, 512 , 3, 1, padding=1)
        self.conv_bn_6 = nn.BatchNorm2d(512)
        torch.nn.init.xavier_normal_(self.conv6.weight)
        torch.nn.init.zeros_(self.conv6.bias)

        self.pool  = nn.MaxPool2d(2,2)

        self.act   = nn.ReLU(inplace=False)
        self.drop = nn.Dropout2d(0.2)    

        self.mlp = nn.Sequential(
            nn.Linear(4 * 4 * 512, 2 * 2 * 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(2 * 2 * 128, 43)     
            #nn.Softmax(dim=-1)    
        )
 
    def forward(self, x):
        x = self.conv_bn_1(self.conv1(x))
        x = self.pool(self.act(x))
        x = self.drop(x)

        x = self.conv_bn_2(self.conv2(x))
        x = self.pool(self.act(x))
        x = self.drop(x)

        x = self.conv_bn_3(self.conv3(x))
        x = self.pool(self.act(x))
        x = self.drop(x)

        x = self.conv_bn_4(self.conv4(x))
        x = self.pool(self.act(x))
        x = self.drop(x)
        
        x = self.conv_bn_5(self.conv5(x))
        x = self.pool(self.act(x))
        x = self.drop(x)
        
        x = self.conv_bn_6(self.conv6(x))
        x = self.pool(self.act(x))
        x = self.drop(x)


        bsz, nch, height, width = x.shape
        x = torch.flatten(x, start_dim=1, end_dim=-1)
        
        y = self.mlp(x)

        return y

In [ ]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', pretrained=False)
model.fc = nn.Linear(512,43)
model.load_state_dict(torch.load("../input/default-gtrsb-models/resnet34.pt"))
model.to(device)

In [ ]:
cnn = SmallCNN()
cnn.load_state_dict(torch.load("../input/default-gtrsb-models/light_cnn_model.pt"))
cnn.to(device)

In [ ]:
#Uncommment for training smallcnn || comment for training rn34
cnn = model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.05, momentum = 0.4)
epochs = 10
steps = 0
running_loss = 0
train_losses = []
test_losses = []
print_every= 100

cnn.train()

In [ ]:
for epoch in range(epochs):
    accuracy = 0
    accuracy_t = 0
    steps = 0
    for inputs, labels in train_loader:
        cnn.train()
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
       
        optimizer.zero_grad()
        
        logps = cnn.forward(inputs)
        loss = criterion(logps, labels)
        

        top_p, top_class = logps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
        train_losses.append(running_loss/len(train_loader))
    with torch.no_grad():
        for inputs_t, labels_t in test_loader:

            steps += 1
            inputs_t, labels_t = inputs_t.to(device), labels_t.to(device)

            logps_t = cnn.forward(inputs_t)

            top_p_t, top_class_t = logps_t.topk(1, dim=1)
            equals_t = top_class_t == labels_t.view(*top_class_t.shape)
            accuracy_t += torch.mean(equals_t.type(torch.FloatTensor)).item()


        print(f"Epoch {epoch+1}/{epochs}.. "
              f"Train accuracy: {accuracy/len(train_loader):.3f}"
              f"Test accuracy: {accuracy_t/len(test_loader):.3f}")
    
        train_losses.append(running_loss/len(train_loader))

In [ ]:
path = "rn34_update.onnx"
path_pt = "rn34_update.pt"
trial = torch.FloatTensor(1,3,256,256).to(device)
torch.onnx.export(cnn, trial, path)
torch.save(cnn.state_dict(), path_pt)